# Part IIA: Explicitly solving Tic-Tac-Toe with the Minimax Algorithm

- To know more about the Minimax algorithm, you can read [the notes I have prepared](../report.pdf) or the [Wikipedia article on it](https://en.wikipedia.org/wiki/Minimax). It is a very general tool for solving decision-making problems, however, it can become very computationally expensive. We will analyze this algorithm for the special case of finite terminating two player games.

- Tic-Tac-Toe is a finite terminating two player game that can only end in a loss for one player (and a win for the other) or in a tie. It can be shown that for every such game, at least one of the players has a non-losing strategy (some of you may see this proof again later in CS 207 - it's basically structural induction over Game Trees). This includes games such as Chess!

- The algorithm is based on forming the game tree: The root of this tree is the starting position, and the children of each node are the positions reachable from the node position. If the game is lost/won at a position, it is terminated, ie, the corresponding node has no children.

- Nodes at even levels (the root is at level 0) are the positions when the first player (call her A) is to play and nodes at the odd levels are the positions when the second player (call her B) is to play.

- Now, we say a node is winning iff there exists a strategy for the player of that node such that no matter what the other player does, she will win. A node is losing iff there if no matter what the player of the node does, the other player can play in such a way that she loses. If neither are true, we say the node is drawn.

- Now, if a node is winning, then there must exist a move the node player makes such that from then on the other player is losing, even if she plays perfectly. A move is a transition from a parent node to a child node (remember parent player and child player are different).

- - So, a node is winning if there exists a child node that is losing.
- - A node is drawn if there exists no losing child node but there does exist a drawn child node.
- - A node is losing if every child node is winning.

- Since we know the statuses of all the leaf nodes (all either winning, losing or drawn), we can get the statuses of every node (ie every position) from the bottom up, recursively. Hence, every node is either winning, losing or drawn, including the root node.

- This proves the nodement above, and this algorithm for finding the node statuses is known as the Minimax algorithm.

- The disadvantage is that this algorithm is extremely computationally expensive. It's training time (time taken to find out status of each node) is proportional to the number of nodes, ie, the number of reachable game nodes. For a game like Chess, this is on the order of 10^40!

- Therefore, the minimax algorithm is not really used, except for tiny problems, and instead, real reinforcement learning algorithms are used, which may be less accurate, but are way less computationally expensive.

In [39]:

import random
import copy
n=int(input("Enter the dimension of the board"))


class GameTree:
    
    def __init__(self, current_player,board=[[0]*n for _ in range(n)] ) -> None:
        self.children={}
        self.board=board
        self.current_player=current_player
        self.best_move=None
        self.status=None

    def play(self,pos,player):
        temp_board=copy.deepcopy(self.board)
        temp_board[pos[0]][pos[1]]=player
        return temp_board

    def win(self):
        won=False
        for i in range(n):
            if(self.board[i][0]!=0 and self.board[i][0]==self.board[i][1] and self.board[i][0]==self.board[i][2]):
                return self.board[i][0]
            if(self.board[0][i]!=0 and self.board[0][i]==self.board[1][i] and self.board[0][i]==self.board[2][i]):
                return self.board[0][i]
        if(self.board[0][0]!=0 and self.board[0][0]==self.board[1][1] and self.board[0][0]==self.board[2][2]):
            return self.board[0][0]
        if(self.board[2][0]!=0 and self.board[2][0]==self.board[1][1] and self.board[2][0]==self.board[0][2]):
            return self.board[2][0]
        if(not won):
           return 0
    
    def moves_left(self):
        return [(i, j) for i in range(n) for j in range(n) if self.board[i][j] == 0]
    
    def find_best_move(self):

        if(self.win()!=0):
            if(self.win()==self.current_player):
                self.status=1
            else:
                self.status=-1
        else:
            if(len(self.moves_left())==0):
                self.status=0
            else:
                found_winning_move=0
                draw=0
                for (x,y) in self.moves_left():
                    
                    child_node=GameTree(-self.current_player,self.play((x,y),self.current_player))
                    
                    self.children[(x,y)]=child_node
                    child_node.find_best_move()
                    if (child_node.status==-1):                       # if any child is losing parent node wins
                        self.status=1
                        self.best_move=(x,y)
                        found_winning_move=1

                    elif(child_node.status==0):
                        if(found_winning_move==0):
                            self.status=0
                            self.best_move=(x,y)
                            draw=1
                    
                    

                
                if(found_winning_move==0 and draw==0):
                    self.status=-1
                    self.best_move=random.choice(list(self.children.keys()))
        
                    

    def __str__(self) -> str:
        output = ''
        for i in range(n):
            for j in range(n):
                if self.board[i][j] == 1:
                    output += ' X  '
                elif self.board[i][j] == -1:
                    output += ' O  '
                else:
                    output += ' -  '
            output += '\n'
        return output

first=input("Do you want to start first. Enter y/n")

board=[[0]*n for _ in range(n)]

if(first=='y'):
    row = int(input("Enter row: "))
    col = int(input("Enter column: "))
    board[row][col]=1
    comp=GameTree(-1,board)
    print(comp)
    comp.find_best_move()
    comp=comp.children[comp.best_move]
    print(comp)

    while True:

        if(comp.win()!=0):
            break
        else:
            if(len(comp.moves_left())==0):
                print("Draw")
                break
        wrong_move=True
        while wrong_move:
            try:

                row = int(input("Enter row: "))
                col = int(input("Enter column: "))
                if board[row][col] == 0:
                    wrong_move = False
                else:
                    print("Invalid move. Try again.")
            except (ValueError, IndexError):
                    print("Invalid input. Try again.")

        comp=comp.children[(row,col)]
        print(comp)
        if(comp.win()!=0):
            break
        else:
            if(len(comp.moves_left())==0):
                print("Draw")
                break
        comp=comp.children[comp.best_move]
        print(comp)

    if(comp.win()==1):
        print("You won")
    elif(comp.win()==-1):
        print("Computer Won")

else:
    comp=GameTree(1,board)   
    comp.find_best_move()
    comp=comp.children[comp.best_move]
    print(comp)
    while True:
        if(comp.win()!=0):
            break
        else:
            if(len(comp.moves_left())==0):
                print("Draw")
                break
        wrong_move=True
        while wrong_move:
            try:

                row = int(input("Enter row: "))
                col = int(input("Enter column: "))
                if board[row][col] == 0:
                    wrong_move = False
                else:
                    print("Invalid move. Try again.")
            except (ValueError, IndexError):
                    print("Invalid input. Try again.")
        comp=comp.children[(row,col)]
        print(comp)
        if(comp.win()!=0):
            break
        else:
            if(len(comp.moves_left())==0):
                print("Draw")
                break
        comp=comp.children[comp.best_move]
        print(comp)

    if(comp.win()==-1):
        print("You won")
    elif(comp.win()==1):
        print("Computer Won")



